# ProfitWell testing

In [112]:
import urllib
import requests # pip install requests 
import calendar;
import time;
import json

headers = {
  'Content-Type': 'application/json',
  'Authorization': '7f68c9b3acf02c17ab97e6681f51f071'
}

## new purchase testing

In [119]:
def make_payload(user, plan, status='active'):
    payload = {
        "user_alias": user['id'],
        "subscription_alias": user['gateway_subscription_id'],
        "email": user['email'],
        "plan_id": plan['id'],
        "plan_interval": plan['interval'],
        "plan_currency": plan['currency'],
        "status": status,
        "value": plan['price'],
        "effective_date": calendar.timegm(time.gmtime())
    }
    return payload


def make_single_payload(user, plan, status='active'):
    payload = {
        "user_alias": user['id'],
        "subscription_alias": user['gateway_subscription_id'],
        "email": user['email'],
        "plan_id": plan['id'],
        "plan_currency": plan['currency'],
        "status": status,
        "value": plan['price'],
        "effective_date": calendar.timegm(time.gmtime()),
        "plan_interval": 'n/a'
    }
    return payload

In [94]:
make_payload(user, plan, 'trialing')

{'user_alias': '2',
 'subscription_alias': '000003',
 'email': 'johnny2_first_buy@test.com',
 'plan_id': '1',
 'plan_interval': 'month',
 'plan_currency': 'usd',
 'status': 'trialing',
 'value': 5000,
 'effective_date': 1617704463}

### Test api status

In [113]:
r = requests.get('https://api.profitwell.com/v2/api-status/', headers=headers)
print(r.status_code, r.text)

200 


### Create subscription

In [89]:
user = {
    "id": "2",
    "email": "johnny2_first_buy@test.com",
    "gateway_subscription_id": '000003'
}

plan = {
    'id': "1",
    'interval': 'month',
    'currency': 'usd',
    'price': 5000
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(make_payload(user, plan)), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_GRBMXQZOL4ZP","subscription_id":"pws_UCLaWCrBgblB","user_alias":"2","subscription_alias":"000003","email":"johnny2_first_buy@test.com","plan_id":"1","plan_interval":"month","plan_currency":"usd","status":"active","value":5000,"effective_date":1617677697,"meta":{}}


### churning subscription

In [92]:
gateway_subscription_id = '000003'
subscription_alians = gateway_subscription_id

r = requests.delete(f'https://api.profitwell.com/v2/subscriptions/{subscription_alians}/?effective_date={calendar.timegm(time.gmtime())}&churn_type=voluntary', headers=headers)
print(r.status_code, r.text)

400 {"non_field_errors":["This subscription is already scheduled to churn. To move the churn date, you must un-churn this subscription first"]}


## testing scenario
1. trail -> month
1. trail -> month -> discount
1. trail -> cancel
1. year
1. year -> discount
1. year -> cancel
1. single
1. single -> another single
1. single -> trail
1. single -> year


### trail -> month

In [95]:
# buy trail
user = {
    "id": "3",
    "email": "trail_to_month@test.com",
    "gateway_subscription_id": 'trail_to_month_01'
}

plan = {
    'id': "13",
    'interval': 'month',
    'currency': 'usd',
    'price': 0
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(make_payload(user, plan, 'trialing')), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_MeJZeNftF9xn","subscription_id":"pws_i9H48QdUL1G9","user_alias":"3","subscription_alias":"trail_to_month_01","email":"trail_to_month@test.com","plan_id":"13","plan_interval":"month","plan_currency":"usd","status":"trialing","value":0,"effective_date":1617704495,"meta":{}}


In [98]:
# trail renew to month
user = {
    "id": "3",
    "email": "trail_to_month@test.com",
    "gateway_subscription_id": 'trail_to_month_id_3_plan_3'
}

plan = {
    'id': "3",
    'interval': 'month',
    'currency': 'usd',
    'price': 3995
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(make_payload(user, plan)), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_MeJZeNftF9xn","subscription_id":"pws_Y3IlydC6m4Na","user_alias":"3","subscription_alias":"trail_to_month_id_3_plan_3","email":"trail_to_month@test.com","plan_id":"3","plan_interval":"month","plan_currency":"usd","status":"active","value":3995,"effective_date":1617758678,"meta":{}}


### trail -> month -> discount

In [110]:
user = {
    "id": "4",
    "email": "trail_to_month_4@test.com",
    "gateway_subscription_id": 'trail_to_month_04'
}

plan = {
    'id': "13",
    'interval': 'month',
    'currency': 'usd',
    'price': 0
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(make_payload(user, plan, 'trialing')), headers=headers)
print(r.status_code, r.text)

400 {"non_field_errors":["Subscription pws_PVLeyFE0od6C already exists"]}


In [99]:
# trail renew to month
user = {
    "id": "4",
    "email": "trail_to_month_4@test.com",
    "gateway_subscription_id": 'trail_to_month_id_4_plan_3'
}

plan = {
    'id': "3",
    'interval': 'month',
    'currency': 'usd',
    'price': 3995
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(make_payload(user, plan)), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_3ndAMCl2gpr8","subscription_id":"pws_KljZdhe1NV6V","user_alias":"4","subscription_alias":"trail_to_month_id_4_plan_3","email":"trail_to_month_4@test.com","plan_id":"3","plan_interval":"month","plan_currency":"usd","status":"active","value":3995,"effective_date":1617758795,"meta":{}}


In [116]:
# month to discount
gateway_subscription_id = 'trail_to_month_id_4_plan_3'
subscription_alians = gateway_subscription_id

payload = {
    'plan_id': "15",
    'interval': 'month',
    "status": "active",
    'value': 995,
    "effective_date": calendar.timegm(time.gmtime())
}

r = requests.put(f'https://api.profitwell.com/v2/subscriptions/{subscription_alians}/', data=json.dumps(payload), headers=headers)
print(r.status_code, r.text)

200 {"user_id":"pwu_3ndAMCl2gpr8","subscription_id":"pws_KljZdhe1NV6V","user_alias":"4","subscription_alias":"trail_to_month_id_4_plan_3","email":"trail_to_month_4@test.com","plan_id":"15","plan_interval":"month","plan_currency":"usd","status":"active","value":995,"effective_date":1617770348,"meta":{}}


### trail -> cancel

In [101]:
user = {
    "id": "6",
    "email": "trail_to_cancel_6@test.com",
    "gateway_subscription_id": 'trail_to_cancel_id_6_plan_13'
}

plan = {
    'id': "13",
    'interval': 'month',
    'currency': 'usd',
    'price': 0
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(make_payload(user, plan, 'trialing')), headers=headers)
print(r.status_code, r.text)

201 {"user_id":"pwu_Aef33LxBYct9","subscription_id":"pws_1IWObI6vDtgz","user_alias":"6","subscription_alias":"trail_to_cancel_id_6_plan_13","email":"trail_to_cancel_6@test.com","plan_id":"13","plan_interval":"month","plan_currency":"usd","status":"trialing","value":0,"effective_date":1617758927,"meta":{}}


In [102]:
# trail to cancel
gateway_subscription_id = 'trail_to_cancel_id_6_plan_13'
subscription_alians = gateway_subscription_id

r = requests.delete(f'https://api.profitwell.com/v2/subscriptions/{subscription_alians}/?effective_date={calendar.timegm(time.gmtime())}&churn_type=voluntary', headers=headers)
print(r.status_code, r.text)

200 {"user_id":"pwu_Aef33LxBYct9","subscription_id":"pws_1IWObI6vDtgz","user_alias":"6","subscription_alias":"trail_to_cancel_id_6_plan_13","email":"trail_to_cancel_6@test.com","plan_id":"13","plan_interval":"month","plan_currency":"usd","status":"churned_voluntary","value":0,"effective_date":1617761422,"meta":{"churn_type":"voluntary"}}


### test add trait

In [104]:
# month
user = {
    "id": "7",
    "email": "month_7@test.com",
    "gateway_subscription_id": 'trail_to_month_id_7_plan_3'
}

plan = {
    'id': "3",
    'interval': 'month',
    'currency': 'usd',
    'price': 3995
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(make_payload(user, plan)), headers=headers)
print(r.status_code, r.text)



201 {"user_id":"pwu_lBFWP4Xo5x8Y","subscription_id":"pws_XvL9as3krtAV","user_alias":"7","subscription_alias":"trail_to_month_id_7_plan_3","email":"month_7@test.com","plan_id":"3","plan_interval":"month","plan_currency":"usd","status":"active","value":3995,"effective_date":1617763883,"meta":{}}


In [107]:
traits = [
    {
        "email": "month_7@test.com",
        "category": "card type",
        "trait": "visa"
    },
    {
        "email": "month_7@test.com",
        "category": "first form", #what is first form
        "trait": "form name"
    },
    {
        "email": "month_7@test.com",
        "category": "gateway name",
        "trait": "ccbill_internal_v2"
    },
    {
        "email": "month_7@test.com",
        "category": "utm source", #what is utm source
        "trait": "source"
    },
]
for i in range(1, 10):
    for trait in traits:
        r = requests.put('https://api.profitwell.com/v2/customer_traits/trait/', data = json.dumps(trait), headers=headers)
        print(r.status_code, r.text)
        

400 {"params":"Customer already has trait (category: 'card type', trait: 'visa')"}
400 {"params":"Customer already has trait (category: 'first form', trait: 'form name')"}
400 {"params":"Customer already has trait (category: 'gateway name', trait: 'ccbill_internal_v2')"}
400 {"params":"Customer already has trait (category: 'utm source', trait: 'source')"}
400 {"params":"Customer already has trait (category: 'card type', trait: 'visa')"}
400 {"params":"Customer already has trait (category: 'first form', trait: 'form name')"}
400 {"params":"Customer already has trait (category: 'gateway name', trait: 'ccbill_internal_v2')"}
400 {"params":"Customer already has trait (category: 'utm source', trait: 'source')"}
400 {"params":"Customer already has trait (category: 'card type', trait: 'visa')"}
400 {"params":"Customer already has trait (category: 'first form', trait: 'form name')"}
400 {"params":"Customer already has trait (category: 'gateway name', trait: 'ccbill_internal_v2')"}
400 {"params

### single document

In [118]:
user = {
    "id": "8",
    "email": "single_8@test.com",
    "gateway_subscription_id": 'single_id_8_plan_4'
}

plan = {
    'id': "4",
    'currency': 'usd',
    'price': 5995
}

r = requests.post('https://api.profitwell.com/v2/subscriptions/', data = json.dumps(make_single_payload(user, plan)), headers=headers)
print(r.status_code, r.text)

400 {"plan_interval":["This field is required."]}


In [ ]:
O